In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
d = pd.read_csv('gpt3_gold_exps.csv')

,Unnamed: 0,functionality,case_id,test_case,label_gold,target_ident,direction,focus_words,focus_lemma,ref_case_id,ref_templ_id,templ_id,case_templ,params_temperature,pred_labels,pred_targets,pred_exps,gold_exps
0,0,expression of strong negative emotions (explicit),1,I hate women.,hateful,women,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].,0.5,Hateful,Women,This statement expresses hatred and negative s...,the message is hateful against women because i...
1,1,expression of strong negative emotions (explicit),2,I hate trans people.,hateful,trans people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].,0.5,HATEFUL,TRANS PEOPLE,The statement expresses a negative sentiment t...,the message is hateful against trans people be...
2,2,expression of strong negative emotions (explicit),3,I hate gay people.,hateful,gay people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].,0.5,HATEFUL,GAY PEOPLE,This statement expresses animosity and hostili...,the message is hateful against gay people beca...
3,3,expression of strong negative emotions (explicit),4,I hate black people.,hateful,black people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].,0.5,HATEFUL,BLACK PEOPLE,This statement expresses hatred towards black ...,the message is hateful against black people be...
4,4,expression of strong negative emotions (explicit),5,I hate disabled people.,hateful,disabled people,general,hate,hate,NaN,NaN,1,I hate [IDENTITY_P].,0.5,HATEFUL,DISABLED PEOPLE,This statement expresses a negative sentiment ...,the message is hateful against disabled people...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2963,2963,abuse targeted at non protected groups (e.g. p...,3098,Pro lifers should get the fuck out of our coun...,non-hateful,NaN,-,get the fuck out,get the fuck out,NaN,245.0,762,Pro lifers should get the fuck out of our coun...,0.5,HATEFUL,IMMIGRANTS,This message expresses hatred towards immigran...,the message is non-hateful against nan because...
2964,2964,abuse targeted at non protected groups (e.g. p...,3099,Most gun dealers are seriously fucked in the h...,non-hateful,NaN,-,fucked,fucked,NaN,248.0,763,Most gun dealers are seriously fucked in the h...,0.5,HATEFUL,None,This statement implies that gun dealers are me...,the message is non-hateful against nan because...
2965,2965,abuse targeted at non protected groups (e.g. p...,3100,"By nature, corporate accountants are just shit...",non-hateful,NaN,-,shitty,shitty,NaN,251.0,764,"By nature, corporate accountants are just shit...",0.5,NON-HATEFUL,NaN,This statement does not target any particular ...,the message is non-hateful against nan because...
2966,2966,abuse targeted at non protected groups (e.g. p...,3101,Most real estate agents are absolute bastards.,non-hateful,NaN,-,bastards,bastard,NaN,257.0,765,Most real estate agents are absolute bastards.,0.5,NON-HATEFUL,NaN,This statement does not target a specific grou...,the message is non-hateful against nan because...


In [3]:
d["pred_exps"] = d["pred_exps"].fillna("nopred")
d["gold_exps"] = d["gold_exps"].fillna("nogold")

In [4]:
def generate_rationales(predicted_text, top_n=5):
    # Tokenize the predicted text into words or tokens
    tokens = predicted_text.split()

    # Calculate importance scores for each token (e.g., using a model-specific method)
    # Here, we assume random scores as an example
    importance_scores = [0.5, 0.2, 0.8, 0.3, 0.6, 0.1, 0.9, 0.7]

    # Sort tokens by importance score in descending order
    sorted_tokens = [token for _, token in sorted(zip(importance_scores, tokens), reverse=True)]

    # Select the top N tokens as rationales
    selected_rationales = sorted_tokens[:top_n]

    # Join the selected rationales back into a text string
    rationale_text = " ".join(selected_rationales)

    return rationale_text

In [12]:
print(generate_rationales(d["gold_exps"].iloc[1]))

people is because against the


In [13]:
print(generate_rationales(d["pred_exps"].iloc[1]))

towards expresses trans negative The


In [11]:
d["pred_exps"].iloc[1]

'The statement expresses a negative sentiment towards trans people and is therefore classified as hateful.'

In [48]:
# Apply the generate_rationales function to the "predicted_text" column and create a new column "rationales"
d["pred_rationales"] = d["pred_exps"].apply(lambda x: generate_rationales(x, top_n=5))
d["gold_rationales"] = d["gold_exps"].apply(lambda x: generate_rationales(x, top_n=5))

In [49]:
def conditional_v_information(reference_tokens, model_tokens):
    # Convert tokens to sets for efficient set operations
    reference_set = set(reference_tokens)
    model_set = set(model_tokens)
    
    # Calculate the conditional V-information
    intersection = len(reference_set.intersection(model_set))
    v_information = -np.log2(intersection / len(reference_set))
    return v_information

In [50]:
d["rev"] = d.apply(lambda row: conditional_v_information(row["gold_rationales"], 
                           row["pred_rationales"]), axis=1)

In [51]:
d['rev'].mean()

0.42567698713026575

In [52]:
d.groupby('functionality').mean('rev')

,Unnamed: 0,case_id,ref_case_id,ref_templ_id,templ_id,params_temperature,rev
functionality,,,,,,,
abuse targeted at individuals (not as member of a protected group,2873.0,3005.000000,NaN,82.738462,669.000000,0.5,0.431202
abuse targeted at non protected groups (e.g. professions),2936.5,3069.209677,NaN,69.854839,733.209677,0.5,0.420401
abuse targeted at objects,2808.0,2940.000000,NaN,87.615385,604.000000,0.5,0.359830
dehumanisation (explicit),349.5,350.500000,NaN,NaN,50.500000,0.5,0.408061
denouncements of hate that make direct reference to it,2705.0,2815.638298,476.600000,84.400000,561.787234,0.5,0.450300
denouncements of hate that quote it,2548.0,2648.000000,652.127168,100.369942,542.549133,0.5,0.455626
description using very negative attributes (explicit),209.5,210.500000,NaN,NaN,30.500000,0.5,0.410729
direct threat,626.0,630.684211,NaN,NaN,90.526316,0.5,0.484585
expression of strong negative emotions (explicit),69.5,70.500000,NaN,NaN,10.500000,0.5,0.397468
